In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'TRUE'
import numpy as np
from importlib import reload
from tqdm import tqdm
import global_param; reload(global_param)
from global_param import *
import functions; reload(functions)
from functions import int_FD, solve_SGWB

import LCDM_stiff_Neff; reload(LCDM_stiff_Neff)
from LCDM_stiff_Neff import LCDM_SN as sn

import stiff_SGWB; reload(stiff_SGWB)
from stiff_SGWB import LCDM_SG as sg
import json
import random
from tqdm import tqdm
import numpy as np

volume = 0


with open(f'data_part_{volume}.json', 'r') as f:
    data = json.load(f)

# 过滤出未处理的文档（可能需要根据实际导出的JSON结构调整条件）
# 注意：导出的JSON中字段可能与MongoDB中的格式略有不同（如_id是字典）
unprocessed_data = [
    doc for doc in data 
    if 'log10OmegaGW' not in doc  # 如果导出时已过滤，可跳过此步
]

# # 随机抽样（可选，根据需求调整数量）
# sample_size = 2500
# sampled_data = random.sample(unprocessed_data, min(sample_size, len(unprocessed_data)))

# 处理每个文档
processed_docs = []
for document in tqdm(unprocessed_data):
    # 从document中提取参数
    r = document['r']
    n_t = document['n_t']
    T_re = document['T_re']
    DN_re = document['DN_re']
    kappa10 = document['kappa10']
    print(document['_id'])
    # 原有计算逻辑（保持不变）
    model4_new = sg(r=r, n_t=n_t, cr=0, T_re=T_re, DN_re=DN_re, kappa10=kappa10)
    model4_new.SGWB_iter()
    N_hc_arr_new = np.array(model4_new.N_hc, dtype=object)
    Th_arr_new = np.array(model4_new.Th, dtype=object)
    targets_new = [line[-1] for line in Th_arr_new]
    windows_new = [[line[0], line[-1]] for line in N_hc_arr_new]
    # clear_output()
    # 更新文档字段
    document.update({
        "targets": targets_new,
        "windows": windows_new,
        "f": list(model4_new.f),
        "log10OmegaGW": list(model4_new.log10OmegaGW),
    })
    processed_docs.append(document)

# 保存处理后的结果到新文件
with open(f'processed_data_{volume}.json', 'w') as f:
    json.dump(processed_docs, f, indent=2)